In [1]:
import re
import numpy as np
import glob

In [12]:
filenames = glob.glob('data/*txt')
with open('data/all_lyrics.txt', 'w') as outfile:
    for fname in filenames:
        if 'all_lyrics' in fname: continue
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

with open('data/all_lyrics.txt', 'r') as f:
    text = f.read()

with open('data/all_lyrics.txt', 'w') as f:
    list_of_chars = "[^\n\"$&\'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWY\[\]abcdefghijklmnopqrstuvwxyz ]"
    regex = re.compile(list_of_chars)
    f.write(regex.sub('', text))

In [13]:
with open('data/all_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

308423


In [14]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 "$&'(),-./012345789:?ABCDEFGHIJKLMNOPQRSTUVWY[]abcdefghijklmnopqrstuvwxyz
75


In [15]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi there'))
print(decode(encode('hi there')))

[56, 57, 1, 68, 56, 53, 66, 53]
hi there


In [16]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([308423]) torch.int64
tensor([ 2, 41, 61, 63, 59, 53,  1, 41, 57, 55])


In [17]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 8
train_data[:block_size+1]

tensor([ 2, 41, 61, 63, 59, 53,  1, 41, 57])

In [19]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('------')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} target is {target}')

inputs:
torch.Size([4, 8])
tensor([[57, 67,  1,  0, 37, 66,  1, 62],
        [63, 67, 68,  1, 63, 54,  1, 73],
        [68, 69, 52, 53,  2,  0,  0, 23],
        [ 1, 31,  5, 60, 60,  1, 66, 53]])
targets:
torch.Size([4, 8])
tensor([[67,  1,  0, 37, 66,  1, 62, 63],
        [67, 68,  1, 63, 54,  1, 73, 63],
        [69, 52, 53,  2,  0,  0, 23, 60],
        [31,  5, 60, 60,  1, 66, 53, 49]])
------
when input is [57] target is 67
when input is [57, 67] target is 1
when input is [57, 67, 1] target is 0
when input is [57, 67, 1, 0] target is 37
when input is [57, 67, 1, 0, 37] target is 66
when input is [57, 67, 1, 0, 37, 66] target is 1
when input is [57, 67, 1, 0, 37, 66, 1] target is 62
when input is [57, 67, 1, 0, 37, 66, 1, 62] target is 63
when input is [63] target is 67
when input is [63, 67] target is 68
when input is [63, 67, 68] target is 1
when input is [63, 67, 68, 1] target is 63
when input is [63, 67, 68, 1, 63] target is 54
when input is [63, 67, 68, 1, 63, 54] target is 1
w

In [20]:
print(xb)

tensor([[57, 67,  1,  0, 37, 66,  1, 62],
        [63, 67, 68,  1, 63, 54,  1, 73],
        [68, 69, 52, 53,  2,  0,  0, 23],
        [ 1, 31,  5, 60, 60,  1, 66, 53]])


In [23]:
print(yb)

tensor([[67,  1,  0, 37, 66,  1, 62, 63],
        [67, 68,  1, 63, 54,  1, 73, 63],
        [69, 52, 53,  2,  0,  0, 23, 60],
        [31,  5, 60, 60,  1, 66, 53, 49]])


### Bigram model

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next torkne from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx: (B, T) 
        logits = self.token_embedding_table(idx) # (B, T = 8, C = channels == vocab size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T = 8 for the 8 context) indices of current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # now (B, C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m  = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 75])
tensor(5.0185, grad_fn=<NllLossBackward0>)

RjEBMArY0T.i?B2'2Tn&KnLhA[UBR[YD&AW$0LTMpup3Bj2UsenM4P2'1A)Ki:3vFjHd$Ir9Oim&iz?3-QVgJre4w92B'Y[iC,[)


In [32]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for steps in range(10000): 
    # sample new batch of data 
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3260750770568848


In [49]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


We an frtheeldoutoun sdeyo I as 
Yo f cu gestht waberan'th e Wh taitr on ju fr s m as s f ur oms ane yold
Whasouman besethtove oucth ainea lomigile f be t waby'me
Sore 
Se "
Angitourongid pll w ieth cispl he twhunkngandou, oucal y?
Oht o achery y the
Goditr s siletoulg, tre a teeskild
"
I we
Whorasoup s don bou me
"Re 
I plous f al het
Laknd les nd atoll u'
't wan fayoforou, y, w fout
Anke I'tod dito..
"
Th-at manndinsoverve.. g I wevethevean, olstow
'rekendos y y Landlat'low
Hend ndonond

y y
W


## Self attention

In [57]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [58]:
# we want x[b, t] = mean_i<=t{}
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [68]:
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [72]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei@x
torch.allclose(xbow, xbow3)

True

In [59]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [62]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a@b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
--


In [85]:
# self attention 

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, head_size)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [86]:
out

tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9067e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8599e-01,  2.8623e-01,
           5.7099e-01],
         [ 4.3974e-01, -1.4227e-01, -1.3157e-01,  2.8895e-03, -1.3222e-01,
           6.6093e-04, -2.7904e-01, -2.2676e-01, -2.8723e-01,  5.7456e-01,
           5.6053e-01, -2.5208e-01,  9.7243e-02,  1.0771e-01,  3.0455e-02,
           1.0727e+00],
         [ 4.3615e-01, -6.6358e-02, -2.9296e-01,  7.4315e-02,  5.4381e-02,
          -7.0388e-02, -6.8984e-02, -8.2153e-02, -2.9377e-01, -5.8952e-02,
           3.5887e-01, -2.3088e-03, -1.8212e-01, -3.6143e-02, -6.7189e-02,
           1.1412e+00],
         [ 4.2068e-01, -1.0619e-01, -2.9984e-01,  5.2820e-02,  2.0077e-01,
          -1.6048e-01, -3.5710e-02, -8.3110e-02, -1.7919e-01,  7.7992e-02,
           1.2719e-01,  2.2611e-02, -5.1810e-02,  7.4466e-02,  1.8131e-01,
           8.4463e-01],
    

In [ ]:
with open('./out/more.txt', 'w+') as f:
    f.write('.')